## Importing the elastic search modules

In [3]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
es = Elasticsearch()
import json,requests

### I sent a GET request to my local host to verify that the index- 'ericsson' has indeed been created.

In [2]:
res = requests.get('http://localhost:9200/ericsson')
print (res.content)
es = Elasticsearch([{'ost': 'localhost', 'port': '9200'}])

b'{"ericsson":{"aliases":{},"mappings":{"properties":{"file_loc":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"keywords":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"summary":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}}}},"settings":{"index":{"routing":{"allocation":{"include":{"_tier_preference":"data_content"}}},"number_of_shards":"1","provided_name":"ericsson","creation_date":"1607066569976","number_of_replicas":"1","uuid":"jqMLhP2USEaEpmPrIN8ukw","version":{"created":"7100099"}}}}}'


### I then accessed the dataframe and removed any unnecassary elements in the file name

In [4]:
df=pd.read_csv('data.csv',index_col=0)
df['file_loc'] = df['file_loc'].str[2:-3]
for i in range(len(df)):
    df.iloc[i]['keywords'] = df.iloc[i]['keywords'][1:-1].replace("'","")

In [5]:
df

,file_loc,keywords,summary
0,Peri K,"sarthak, concepts, numerical, teams, excellent...","Back Benchers Education 605 - 607, Bestech Ch..."
1,Accomplishments,"vit, school, team, member, finalist, organizat...","Chapter Coordinator, Think For All (2011–2015)..."
2,MAM Application Essays,"data, business, question, management, world, a...",MAM Application Questions Masters in Analyt...
3,Sarthak_MAM Application Essays,"data, business, like, financial, courses, mana...",MAM Application Questions Masters in Analyt...
4,Sarthak_SOP_CMU,"data, technology, academics, experience, advan...",USER ID- bhargavasarthak99@gmail.com The adve...
5,PH_Sarthak,"years, remain, sheer, different, greatly, majo...",Cosmos a spacetime odyssey was a documentary t...
6,Sarthak_SOP_Col,"academic, data, extensive, technology, experie...",Sarthak Bhargava The ubiquitous expansion of ...
7,Sarthak_NTU_Essay_1,"technologies, academic, study, problems, caree...",Sarthak Bhargava The domain of Analytics is b...
8,Essay 2,"business, analytics, career, pioneering, core,...",Sarthak Bhargava My affinity towards numbers ...
9,101120,"data, technology, academics, experience, progr...",The advent of Computers in 1940s drastically c...


I set file_loc as the key so that can be used as an id

In [192]:
df.set_index('file_loc',inplace=True)

In [193]:
dic=df.T.to_dict('dict')

In [194]:
dic['Peri K']

{'keywords': 'sarthak, concepts, numerical, teams, excellent, education, science, vit, kameswaran, methods, time, achievements, differential, individual, opportunities, interpolation, assignments, exceptional, solving',
 'summary': 'Back Benchers Education  605 - 607, Bestech Chambers,  Radisson Blu, Gurgaon         LOR Suggestions    In a letter specify:  How does the recommender know the applicant? As a member of my class, he was participative and enhanced the learning of the entire class. Apart from his excellent academic performance, he took part in the annual Math-a-thon which was organized by me and several other SAS faculties.'}

 Pushing the content in the dictionary to the elasticsearch server

In [ ]:
for doc in list(dic.keys()):
    res = es.index(index='ericsson',doc_type='doc',id=doc,body=dic[doc]) 

Sending a GET call to the server to verify the entire process

In [6]:
res=es.get(index='ericsson',doc_type='doc',id='0')

C:\Users\sarth\anaconda3\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [7]:
print(res)

{'_index': 'ericsson', '_type': 'doc', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 2, 'found': True, '_source': {'file_loc': "['Peri K.']", 'keywords': "['sarthak', 'concepts', 'numerical', 'teams', 'excellent', 'education', 'science', 'vit', 'kameswaran', 'methods', 'time', 'achievements', 'differential', 'individual', 'opportunities', 'interpolation', 'assignments', 'exceptional', 'solving']", 'summary': 'Back Benchers Education  605 - 607, Bestech Chambers,  Radisson Blu, Gurgaon         LOR Suggestions    In a letter specify:  How does the recommender know the applicant? As a member of my class, he was participative and enhanced the learning of the entire class. Apart from his excellent academic performance, he took part in the annual Math-a-thon which was organized by me and several other SAS faculties.'}}


We can see the process has been successful and our data can now be accessed from the es database